In [2]:
import torch.nn as nn
from torchsummary import summary
import numpy as np
import math

ModuleNotFoundError: No module named 'torch'

In [167]:
def feature_maps_multiplies(img_size):
        layers_num = int(math.log(img_size, 2) - 2)
        ms = multiplies(layers_num)
        return ms
    

def multiplies(m):
    l = []
    for i in range(m - 1):
        if len(l) > 0:
            l.append(l[i-1] * 2)
        else:
            l.append(2)
    return l    

In [198]:
class Discriminator(nn.Module):
    def __init__(self,
                 img_size,
                 feature_maps_size, 
                 channels_num):
        super(Discriminator, self).__init__()
        
        self.img_size = img_size
        self.ndf = feature_maps_size
        self.nc = channels_num  
        print(img_size, feature_maps_size, channels_num)
        self.main = nn.Sequential(
            self._make_layers(img_size, feature_maps_size, channels_num))
        
#         summary(self.main.cuda(), (self.nc, self.img_size, self.img_size))

#         self.main.apply(weights_init)

    def forward(self, input):
        return self.main(input)
    
    
    def _make_layers(self, img_size, ndf, nc):
    
        layers = []

        layers += [ nn.Conv2d(nc, ndf, 4, 2, 1, bias=False) ]
        layers += [ nn.LeakyReLU(0.2, inplace=True) ]

        multiplies = feature_maps_multiplies(img_size)

        for m in multiplies:
            layers += [ nn.Conv2d(ndf * m // 2, ndf * m, 4, 2, 1, bias=False) ]
            layers += [ nn.BatchNorm2d(ndf * m) ]
            layers += [ nn.LeakyReLU(0.2, inplace=True) ]

        layers += [ nn.Conv2d(ndf * multiplies[-1], 1, 4, 1, 0, bias=False) ]
        layers += [ nn.Sigmoid() ]
    
        return nn.Sequential(*layers)

In [199]:
img_size = 64

In [200]:
d1 = Discriminator(img_size, 64, 3)

64 64 3
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           3,072
         LeakyReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3          [-1, 128, 16, 16]         131,072
       BatchNorm2d-4          [-1, 128, 16, 16]             256
         LeakyReLU-5          [-1, 128, 16, 16]               0
            Conv2d-6            [-1, 256, 8, 8]         524,288
       BatchNorm2d-7            [-1, 256, 8, 8]             512
         LeakyReLU-8            [-1, 256, 8, 8]               0
            Conv2d-9            [-1, 512, 4, 4]       2,097,152
      BatchNorm2d-10            [-1, 512, 4, 4]           1,024
        LeakyReLU-11            [-1, 512, 4, 4]               0
           Conv2d-12              [-1, 1, 1, 1]           8,192
          Sigmoid-13              [-1, 1, 1, 1]               0
Total params: 2,765,568
Trainab

In [190]:
summary(d1.cuda(), (3, img_size, img_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           3,072
         LeakyReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3          [-1, 128, 16, 16]         131,072
       BatchNorm2d-4          [-1, 128, 16, 16]             256
         LeakyReLU-5          [-1, 128, 16, 16]               0
            Conv2d-6            [-1, 256, 8, 8]         524,288
       BatchNorm2d-7            [-1, 256, 8, 8]             512
         LeakyReLU-8            [-1, 256, 8, 8]               0
            Conv2d-9            [-1, 512, 4, 4]       2,097,152
      BatchNorm2d-10            [-1, 512, 4, 4]           1,024
        LeakyReLU-11            [-1, 512, 4, 4]               0
           Conv2d-12              [-1, 1, 1, 1]           8,192
          Sigmoid-13              [-1, 1, 1, 1]               0
Total params: 2,765,568
Trainable param

In [173]:
class Generator(nn.Module):
    def __init__(self,
                 img_size,
                 nz,
                 ngf, 
                 nc):
        super(Generator, self).__init__()
        
        self.img_size = img_size
        self.nz = nz
        self.ngf = ngf
        self.nc = nc
        print(img_size, nz, ngf, nc)
        self.main = self._make_layers(img_size, nz, ngf, nc)
        
#         summary(self.main.cuda(), (self.nz, 1, 1))
        

#         self.main.apply(weights_init)

    def forward(self, input):
        return self.main(input)
    
    
    def _make_layers(self, img_size, nz, ngf, nc):
    
        multiplies = list(reversed(feature_maps_multiplies(img_size)))
        
        print(multiplies)
    
        layers = []

        layers += [ nn.ConvTranspose2d(nz, ngf * multiplies[0], 4, 1, 0, bias=False) ]
        layers += [ nn.BatchNorm2d(ngf * multiplies[0])]
        layers += [ nn.ReLU(True) ]

        for m in multiplies:
            layers += [ nn.ConvTranspose2d(ngf * m, ngf * m // 2, 4, 2, 1, bias=False) ]
            layers += [ nn.BatchNorm2d(ngf * m // 2) ]
            layers += [ nn.ReLU(True) ]

        layers += [ nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False) ]
        layers += [ nn.Tanh() ]
    
        return nn.Sequential(*layers)

In [191]:
nz = 200
img_size = 64

In [192]:
g1 = Generator(img_size, nz, 64, 3)

64 200 64 3
[8, 4, 2]


In [194]:
summary(g1.cuda(), (nz, 1, 1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1            [-1, 512, 4, 4]       1,638,400
       BatchNorm2d-2            [-1, 512, 4, 4]           1,024
              ReLU-3            [-1, 512, 4, 4]               0
   ConvTranspose2d-4            [-1, 256, 8, 8]       2,097,152
       BatchNorm2d-5            [-1, 256, 8, 8]             512
              ReLU-6            [-1, 256, 8, 8]               0
   ConvTranspose2d-7          [-1, 128, 16, 16]         524,288
       BatchNorm2d-8          [-1, 128, 16, 16]             256
              ReLU-9          [-1, 128, 16, 16]               0
  ConvTranspose2d-10           [-1, 64, 32, 32]         131,072
      BatchNorm2d-11           [-1, 64, 32, 32]             128
             ReLU-12           [-1, 64, 32, 32]               0
  ConvTranspose2d-13            [-1, 3, 64, 64]           3,072
             Tanh-14            [-1, 3,